In [75]:
import pandas as pd

In [76]:
df_principals=pd.read_csv("title.principals.csv")

In [77]:
cast = df_principals[df_principals['category'].isin(['actor', 'actress'])]
crew = df_principals[~df_principals['category'].isin(['actor', 'actress'])]
ordered_cast = pd.DataFrame(cast.groupby(by='tconst').apply(lambda x: [x for x in x['nconst']]))
ordered_cast = ordered_cast.reset_index().rename(columns={0: 'cast'})


In [78]:

ordered_crew = pd.DataFrame(crew.groupby(by='tconst').apply(lambda x: [x for x in x['nconst']]))
ordered_crew = ordered_crew.reset_index().rename(columns={0: 'crew'})

In [82]:
#movies_ratings_df= movies_ratings_df.merge(df_ratings, on="tconst", how="inner")

In [83]:
#movies_ratings_df

In [84]:
# If it went right, we should have no duplicated movies (tconst)
#movies_ratings_df.duplicated('tconst').any()

In [85]:
# dropping ALL duplicte values
#movies_ratings_df.drop_duplicates(subset ="tconst",
#                     keep = False, inplace = True)

In [86]:
#movies_ratings_df[movies_ratings_df['tconst']=='tt0088763']

In [87]:
#movies_ratings_df['cast'].fillna('Unknown', inplace=True)
#movies_ratings_df['crew'].fillna('Unknown', inplace=True)


In [88]:
to_wtv = pd.DataFrame(df_principals.groupby(by='tconst').apply(lambda x: [x for x in x['nconst']])).reset_index().iloc[:, 1]
to_wtv.head(3)

0               [nm1588970, nm0005690, nm0374658]
1                          [nm0721526, nm1335271]
2    [nm0721526, nm1770680, nm1335271, nm5442200]
Name: 0, dtype: object

In [89]:
from gensim.models import Word2Vec
import gensim
wtv = Word2Vec(window=10, min_count=5)
wtv.build_vocab(to_wtv)
wtv.train(to_wtv, total_words=wtv.corpus_total_words, epochs=10)


(7429180, 10485750)

In [90]:
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import requests

def get_name(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    return soup.select('.header .itemprop')[0].text

def get_image(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    candidates = soup.select('#name-poster')
    return candidates[0].attrs['src'] if candidates else 'https://m.media-amazon.com/images/G/01/imdb/images/nopicture/medium/name-2135195744._CB466677935_.png'

def render_person(id):
    name = get_name(id)
    picture = get_image(id)
    return f"""
    <div style="width: 150px; text-align: center">
        <h4 style='margin-top: -5px'>{name}</h4>
        <div style='font-size:75%; margin-bottom: 5px'>{id}</div>
        <a href="https://www.imdb.com/name/{id}" target="_blank">
            <img style="width: 100px; display: block; margin-left: auto; margin-right: auto;" src="{picture}"/>
        </a>
    </div>
    """

def show_similars(id, n=10):
    if id in wtv.wv: 
        display(HTML(render_person(id)))
    renders = []
    for similar_id, score in wtv.wv.most_similar(id, topn=n):
        renders.append(render_person(similar_id))
        
    carousel = ''.join(
        [
            f'<div style="margin-left: 10px; float: left">{p}</div>' 
            for p in renders
        ]
        )
    display(HTML(f'<div style="width: 1800px">{carousel}</div>'))

def show_similars_tovector(id, n=10):
    renders = []
    
    for similar_id, score in wtv.wv.most_similar(id, topn=n):
        renders.append(render_person(similar_id))
        
    carousel = ''.join(
        [
            f'<div style="margin-left: 10px; float: left">{p}</div>' 
            for p in renders
        ]
    )
    display(HTML(f'<div style="width: 1800px">{carousel}</div>'))

In [91]:
#Show actors who are similar in popularity

show_similars('nm0000138')

In [92]:
def who_doesnt_match(person1, person2, person3, person4):
    
    p1 = str(person1); p2=str(person2); p3=str(person3); p4=str(person4); 
    result = wtv.wv.doesnt_match([p1, p2, p3, p4])
    
    if result in wtv.wv: 
        display(HTML(render_person(result)))

In [93]:
import random
angelina_jolie= 'nm0001401'
sean_penn = 'nm0000576'
ryan_gosling = 'nm0331516'
random1 = random.sample(list(df_principals.nconst.unique()), 1)[0]
random2 = random.sample(list(df_principals.nconst.unique()), 1)[0]
random3 = random.sample(list(df_principals.nconst.unique()), 1)[0]

In [94]:
print(random1, random2, random3)

nm1726906 nm0216412 nm2251683


In [95]:
paul_newman = 'nm0000056'
joseph_gordon_levitt = 'nm0330687'
jennifer_lawrence = 'nm2225369'
rooney_mara = 'nm1913734'

In [73]:
#Show actors who are not equally famous
who_doesnt_match(paul_newman, joseph_gordon_levitt, jennifer_lawrence, rooney_mara)